<a href="https://colab.research.google.com/github/ignacioaranguren1/bd_2/blob/main/bd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import keras_tuner
import datetime as dt

from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.layers import BatchNormalization

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%matplotlib inline

datapath = r'/content/drive/MyDrive/bd2/data'
os.chdir(datapath)

1.In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers. 

In [6]:
data = pd.read_pickle('returns_chars_panel.pkl')
macro = pd.read_pickle('macro_timeseries.pkl')

In [7]:
def train_validation_test_split(data,train_end_date,validation_end_date):
  tmp = data.reset_index()
  train = tmp[tmp.date<=train_end_date].set_index(['date','permno'],drop=True)
  validation = tmp[(tmp.date>train_end_date) & (tmp.date<=validation_end_date)].set_index(['date','permno'],drop=True)
  test = tmp[tmp.date>validation_end_date].set_index(['date','permno'],drop=True)
  return train,validation,test

In [8]:
data_merged = pd.merge(data,macro,on=['date'])
datelist = list(set(data_merged['date']))
datelist.sort()
data_merged.set_index(['date','permno'],drop=True,inplace=True)

In [9]:
# It is worth mentioning that even though we have set these ratios in order to split the data set the resulting
# weighs are not exactly the same because dates can have more than one observation
train_ratio = 0.5
validation_ratio = 0.25
train_date = datelist[int(len(datelist)*train_ratio)]
validation_date = datelist[int(len(datelist)*(train_ratio+validation_ratio))]
X = data_merged.iloc[:,3:].copy()
y = data_merged['excess_ret'].copy()

In [10]:
X_train,X_validation,X_test = train_validation_test_split(X,train_date,validation_date)
y_train,y_validation,y_test = train_validation_test_split(y,train_date,validation_date)

In [11]:
def keras_model(n_layers, units, learning_rate):
    # Model definition separated from tuner in order to achieve modularity 
    # Build model
    model = Sequential()
    model.add(layers.Input(shape=(105,)))
    # Add layers iteratively and assign a units hyperparam selector
    for i in range(n_layers):
        model.add(BatchNormalization()) # Normalizing before activation seems to yield better results than after
        model.add(layers.Dense(units=units[0][i], activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse')
    return model

class HyperRegressor(keras_tuner.HyperModel):
    def __init__(self, n_layers, *args, **kwargs):
        # Pass all arguments except number of layers to parent
        self.n_layers = n_layers
        super().__init__(*args, **kwargs)

    def build(self, hp):
        # Hyperparameters choices and ranges definition 
        # To increase modularity, we declare units choice for each layer in a list 
        units=[hp.Int(f'units_{i + 1}',min_value=16,max_value=160,step=16) for i in range(self.n_layers)],
        learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        return keras_model(self.n_layers, units, learning_rate)

    def fit(self, hp, model, x, y, validation_data, **kwargs):
        model.fit(x, y, **kwargs)
        x_val, y_val = validation_data
        y_pred = model.predict(x_val)
        # Return a single float to minimize.
        return np.mean((y_pred - y_val)**2)

In [13]:
########################
# CONSTANTS DEFINITION #
########################

MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 3
EPOCHS = 8
BATCH_SIZE = 256

def tune_model(n_layers=2):
  # Early stop if loss does not improve after 3 epochs
  callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  tuner = RandomSearch(
        hypermodel=HyperRegressor(n_layers),
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTION_PER_TRIAL,
        overwrite=True,
        directory='bd_2',
        project_name=f'NN_new_{n_layers}'
  )
  tuner.search(
      X_train.values, 
      y_train.values,
      validation_data=(X_validation.values, y_validation.values),
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[callback]
  )
  return tuner

In [12]:
models = []
parameters = []
tuners = []
for n in range(2,5):
    tuner = tune_model(n)
    parameters.append(tuner.get_best_hyperparameters)
    models.append(tuner.get_best_models(1)[0])
    tuners.append(tuner)

Trial 10 Complete [00h 09m 28s]
default_objective: 0.04740524855275794

Best default_objective So Far: 0.04082417058268804
Total elapsed time: 01h 34m 14s
INFO:tensorflow:Oracle triggered exit


In [26]:
for i in range(3):
  print(tuners[i].get_best_hyperparameters()[0].values)

{'units_1': 144, 'units_2': 64, 'learning_rate': 0.000256153969803113}
{'units_1': 64, 'units_2': 96, 'units_3': 112, 'learning_rate': 0.0013688978213179357}
{'units_1': 16, 'units_2': 48, 'units_3': 128, 'units_4': 144, 'learning_rate': 0.0005677736913894471}


In [13]:
with open('models.pkl','wb') as f:
  pickle.dump(models,f)
with open('tuners.pkl','wb') as f:
  pickle.dump(tuners,f)

INFO:tensorflow:Assets written to: ram://4cefee27-841d-4536-af97-baa27475d903/assets
INFO:tensorflow:Assets written to: ram://96fedd41-dad1-4d05-aea2-d2ce631fd171/assets
INFO:tensorflow:Assets written to: ram://b75f643e-f30a-4748-996b-531cb7f81058/assets


In [17]:
def format_units(buffer_dict):
      # Convert units param to a list of units to match processing formatting
      units = []
      # Check if key is unit, if it is add to list 
      for key, value  in buffer_dict.values.items():
          if 'units' in key:
              units += [value]
      # Crate new dict with correct format 
      best_params = {}
      best_params['units'] = [units]
      best_params['learning_rate'] = buffer_dict['learning_rate']
      return best_params

models_refitted = []
results = {}
for i in range(3):
    # Build and refit model with best params
    callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    best_hps = format_units(tuners[i].get_best_hyperparameters()[0])
    n_layers = len(best_hps['units']) # Get num of hidden layers
    model = keras_model(n_layers, **best_hps) # Rebuild model
    model.fit(X, y, epochs=64, batch_size=256,verbose=True, callbacks=[callback])
    models_refitted.append(model)
    # Evaluate train, val and test 
    train_result = model.evaluate(X_train.values,y_train.values,batch_size=256)
    test_result = model.evaluate(X_test.values,y_test.values,batch_size=256)
    val_result = model.evaluate(X_validation.values,y_validation.values,batch_size=256)
    results[f'NN{i + 1}'] = {'train': train_result, 'validation': val_result, 'test': test_result}

Epoch 1/64
14608/14608 [==============================] - 35s 2ms/step - loss: 0.0554
Epoch 2/64
14608/14608 [==============================] - 32s 2ms/step - loss: 0.0296
Epoch 3/64
14608/14608 [==============================] - 32s 2ms/step - loss: 0.0293
Epoch 4/64
14608/14608 [==============================] - 32s 2ms/step - loss: 0.0291
Epoch 5/64
14608/14608 [==============================] - 32s 2ms/step - loss: 0.0289
Epoch 6/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0288
Epoch 7/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0287
Epoch 8/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0286
Epoch 9/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0285
Epoch 10/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0284
Epoch 11/64
14608/14608 [==============================] - 31s 2ms/step - loss: 0.0283
Epoch 12/64
14608/14608 [===========================

In [18]:
results

{'NN1': {'test': 0.024710332974791527,
  'train': 0.02167060784995556,
  'validation': 0.03952166065573692},
 'NN2': {'test': 0.023484626784920692,
  'train': 0.021047215908765793,
  'validation': 0.037058476358652115},
 'NN3': {'test': 0.023973975330591202,
  'train': 0.021303806453943253,
  'validation': 0.03852330520749092}}

In [18]:
with open('refitted_models_BN.pkl','wb') as f:
  pickle.dump(models_refitted,f)

INFO:tensorflow:Assets written to: ram://badb6e86-648e-47cd-a306-01eaf114f5b5/assets
INFO:tensorflow:Assets written to: ram://b003ddc8-fec3-4294-8f3f-28fbafbf577b/assets
INFO:tensorflow:Assets written to: ram://21149ecd-ca54-4391-b42a-2d8bc73ff3c4/assets


2.Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

In [14]:
with open('models.pkl','rb') as f:
  models = pickle.load(f)
with open('tuners.pkl','rb') as f:
  tuners = pickle.load(f)
with open('refitted_models_BN.pkl','rb') as f:
  models_refitted = pickle.load(f)

In [15]:
def r_squared(y_pred, y_test):
    return 1 - np.sum((y_test - y_pred)**2) / np.sum(y_test**2)

In [15]:
rankings = X_test['mvel1'].groupby(['date']).rank()
top_X_test = X_test.loc[rankings<=1000,:].values
top_y_test = y_test.loc[rankings<=1000,:].values
rankings_reverse = X_test['mvel1'].groupby(['date']).rank(ascending=False)
bottom_X_test = X_test.loc[rankings_reverse<=1000,:].values
bottom_y_test = y_test.loc[rankings_reverse<=1000,:].values

R2_oos_df = pd.DataFrame(columns = ['R2_OOS','R2_OOS_top1000','R2_OOS_low1000'],index = ['NN2','NN3','NN4'])
for i in range(3):
    y_pred_all = models_refitted[i].predict(X_test,batch_size=256).reshape(-1,1)
    y_pred_top = models_refitted[i].predict(top_X_test,batch_size=256).reshape(-1,1)
    y_pred_bottom = models_refitted[i].predict(bottom_X_test,batch_size=256).reshape(-1,1)
    
    row = [r_squared(y_pred_all, y_test)[0],
           r_squared(y_pred_top, top_y_test),
           r_squared(y_pred_bottom, bottom_y_test)]
    
    R2_oos_df.iloc[i]= row

In [16]:
R2_oos_df * 100

,R2_OOS,R2_OOS_top1000,R2_OOS_low1000
NN2,11.614579,7.251403,21.135385
NN3,5.597416,4.051352,8.421495
NN4,15.510674,10.886926,26.439364


In [17]:
with open('r_squared.pkl','wb') as f:
  pickle.dump(R2_oos_df,f)

3.Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 


In [12]:
with open('refitted_models_BN.pkl','rb') as f:
  models_refitted = pickle.load(f)
with open('r_squared.pkl','rb') as f:
  R2_oos_df = pickle.load(f)

In [13]:
# X_test.groupby(pd.Grouper(freq='M'))['mvel1'].rank()
best_r_squared = max(R2_oos_df.iloc[:,0])
best_r_squared * 100

15.510674225049337

In [ ]:
feature_importance = {}
for column_name in tqdm(X_train.columns):
    X_tmp = X_test.copy()
    X_tmp[column_name] = 0
    y_pred_temp = models_refitted[2].predict(X_tmp, batch_size=256).reshape(-1,1)
    feature_importance[column_name] = best_r_squared - r_squared(y_pred_temp, y_test)[0]


 67%|██████▋   | 70/105 [07:18<04:47,  8.21s/it]

In [ ]:
importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['magnitude'])
# Retrieve 10 most significant
df_10 = importance_df.sort_values('magnitude').iloc[(len(importance_df['magnitude']) - 10):,]

df_10.index.values

In [ ]:
# Display chart
plt.figure(figsize=(20,5))
plt.barh(np.arange(len(df_10)),df_10.magnitude.values)
plt.yticks(np.arange(len(df_10)),df_10.index.values)
plt.title('Feature Importance',size=16)
plt.show()

b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.

4.Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

5.Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?